# Welcome to the Lab 🥼🧪

## Market Supply & Demand

Let's review supply, demand

Let's get into the lab and take a peak at whats going on.

**Note** This notebook will work with any of the 70k+ markets supported by the Parcl Labs API.

As a reminder, you can get your Parcl Labs API key [here](https://dashboard.parcllabs.com/signup) to follow along. 

In [0]:
import os
import sys
import subprocess
from datetime import datetime

if "google.colab" in sys.modules:
    from google.colab import userdata
    %pip install parcllabs plotly kaleido
    api_key = userdata.get('PARCL_LABS_API_KEY')
elif 'DATABRICKS_RUNTIME_VERSION' in os.environ:
    print("Running in Databricks environment.")
    api_key = dbutils.secrets.get('pricefeed-api', 'PARCL_LABS_API_KEY')  # assuming API key is set in Databricks secrets
    %pip install parcllabs plotly kaleido
else:
    api_key = os.getenv('PARCL_LABS_API_KEY')

In [0]:
import parcllabs
import pandas as pd
import plotly.express as px
from parcllabs import ParclLabsClient

print(f"Parcl Labs Version: {parcllabs.__version__}")

In [0]:
# charting setup
# set some constants for the analysis, such as the logo for the chart, we have a white and blue version
labs_logo_lookup = {
    'blue': 'https://parcllabs-assets.s3.amazonaws.com/powered-by-parcllabs-api.png',
    'white': 'https://parcllabs-assets.s3.amazonaws.com/powered-by-parcllabs-api-logo-white+(1).svg',
    'x': 'https://parcllabs-assets.s3.amazonaws.com/Parcl-x-ParclLabs+API_White.svg'
}

# set charting constants
labs_logo_dict = dict(
        source=labs_logo_lookup['white'],
        xref="paper",
        yref="paper",
        x=0.5,  # Centering the logo below the title
        y=1.02,  # Adjust this value to position the logo just below the title
        sizex=0.15, 
        sizey=0.15,
        xanchor="center",
        yanchor="bottom"
)

# define image dimentions
media_img_size_lookup = {
    'X': {
        'width': 1600,
        'height': 900
    }
}

# optimize the visual for the platform in this case X
PLATFORM = 'X'

# set image sizes
IMG_WIDTH = media_img_size_lookup[PLATFORM]['width']
IMG_HEIGHT = media_img_size_lookup[PLATFORM]['height']

# plotting title settings
PLOT_TITLE_SETTINGS = {
        'y':0.97,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'
    }

In [0]:
client = ParclLabsClient(api_key=api_key)

In [0]:
# we are looking at homes in San Diego, so we look for the name and the type of market
results = client.search_markets.retrieve(
    sort_by='PARCL_EXCHANGE_MARKET',
    as_dataframe=True,
    params={
        'limit': 14
    }
)

trade_parcl_ids = results['parcl_id'].tolist()

### Supply and Demand Activity

Let's review supply and demand drivers

In [0]:
# Prices
supply_demand = client.market_metrics_housing_event_counts.retrieve(
    parcl_id=5826765,
    as_dataframe=True,
    params={
        'limit': 36
    },
    start_date='2023-01-01'
)

supply_demand.head(1)

In [0]:
import plotly.graph_objects as go
import os

# Example data
months = supply_demand['date'].tolist()
sales = supply_demand['sales'].tolist()
new_listings = supply_demand['new_listings_for_sale'].tolist()

# Create figure
fig = go.Figure()

# Add bar for New Listings
fig.add_trace(go.Bar(
    x=months,
    y=new_listings,
    name='New Listings',
    marker_color='#DCECFF'
))

# Add bar for Sales (inverted to appear negative)
fig.add_trace(go.Bar(
    x=months,
    y=[-x for x in sales],
    name='Sales',
    marker_color='#04428C'
))

# Update layout
fig.update_layout(
    title='US Home Sales and New Listings',
    title_x=0.5,  # Centering the title
    xaxis_tickfont_size=14,
    yaxis=dict(
        title='Number of Transactions',
        titlefont_size=16,
        tickfont_size=14,
        # Auto range adjustment for y-axis
        autorange=True
    ),
    height=800,  # Update according to your constant or preference
    width=1200,   # Update according to your constant or preference
    legend=dict(
        x=0.01,
        y=0.99,
        bgcolor='rgba(255, 255, 255, 0)',
        bordercolor='rgba(255, 255, 255, 0)'
    ),
    barmode='overlay',
    bargap=0.15, # gap between bars of adjacent location coordinates
    bargroupgap=0.1, # gap between bars of the same location coordinates
    plot_bgcolor='#080D16', 
    paper_bgcolor='#080D16',
    font=dict(color='#FFFFFF')
)

# Assuming 'labs_logo_dict' is defined elsewhere in your code
# Uncomment and configure the following line if you have a logo to add:
fig.add_layout_image(labs_logo_dict)

# Adding gridlines
fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='LightGrey')
fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='LightGrey')

# Show the plot
fig.show()


Now lets convert this to a ratio, and a table to measure this in a comparable way across many markets. 

new listings/sales

In [0]:
# lets do top 50 metros

results = client.search_markets.retrieve(
    sort_by='TOTAL_POPULATION',
    as_dataframe=True,
    location_type='CBSA',
    params={
        'limit': 50
    }
)

ids = results['parcl_id'].tolist()

In [0]:
supply_demand = client.market_metrics_housing_event_counts.retrieve_many(
    parcl_ids=ids,
    as_dataframe=True,
    params={
        'limit': 36
    },
    start_date='2023-03-01'
)

supply_demand.head()

In [0]:
supply_demand['ratio'] = round(supply_demand['new_listings_for_sale']/supply_demand['sales'], 2)
supply_demand.head()

In [0]:
chart = pd.merge(supply_demand, results[['parcl_id', 'name']], on='parcl_id')

In [0]:
def format_names(nme):
    metro = nme.split(',')[0].split('-')[0].strip()
    metro = metro.split('/')[0].strip()
    return metro

chart['name'] = chart['name'].apply(lambda x: format_names(x))

In [0]:
chart.head()

In [0]:
chart = chart.loc[chart['name'] != 'Austin']
chart = chart.loc[chart['name'] != 'Memphis']
charting_data = chart.pivot_table(values='ratio', index='name', columns='date')
charting_data = charting_data.sort_values('2024-03-01', ascending=False)


In [0]:
import plotly.graph_objects as go
import pandas as pd
import numpy as np
from datetime import datetime

# Define a function to scale the color based on the value
def color_scale(value):
    # Normalize the value to the new range [0, 2] with a midpoint at 1
    if value < 1:
        # Scale from light blue to blue as value approaches 1 from 0
        normalized = value  # No need to normalize since it's already 0 to 1
        r = int(173 + (65 - 173) * normalized)  # Decrease red channel towards darker blue
        g = int(216 + (105 - 216) * normalized)  # Decrease green channel towards darker blue
        b = int(230 + (225 - 230) * normalized)  # Slightly increase blue channel towards 225
    else:
        # Scale from blue to dark blue as value goes from 1 to 2
        normalized = (value - 1) / 1  # Normalize the upper half from 0 to 1
        r = int(65 + (0 - 65) * normalized)  # Decrease red channel towards darkest blue
        g = int(105 + (0 - 105) * normalized)  # Decrease green channel towards darkest blue
        b = int(225 + (139 - 225) * normalized)  # Decrease blue channel towards darker blue

    # Ensure RGB values are within the valid range
    r = max(0, min(255, r))
    g = max(0, min(255, g))
    b = max(0, min(255, b))

    return f'rgb({r},{g},{b})'

# Define a function to format the date
def format_date(date):
    date = datetime.strptime(date, '%Y-%m-%d')
    return date.strftime('%b `' + date.strftime('%y'))

dates = charting_data.columns.tolist()

In [0]:
import plotly.graph_objects as go

# Assuming 'charting_data', 'dates', 'format_date', 'color_scale', and 'labs_logo_dict' are defined

# Define the table
fig = go.Figure(
    data=[go.Table(
        header=dict(
            values=['<b>Market</b>'] + [f"<b>{format_date(date)}</b>" for date in dates],
            fill_color='#080D16',
            font=dict(color='#FFFFFF', size=12),
            align='center',
            height=30
        ),
        cells=dict(
            values=[[f"{name}" for name in charting_data.index]] + 
                   [charting_data[date] for date in dates],
            fill=dict(
                color=[
                    ['#080D16']*len(charting_data.index)] +
                    [[color_scale(charting_data.at[market, date]) for market in charting_data.index] for date in dates]
            ),
            font=dict(
                color='#FFFFFF',
                size=11  # Uniform size for all cells
            ),
            align='center',
            height=30
        ),
        columnwidth=[125] + [100] * len(dates)  # Adjust these values based on your needs
    )]
)

# Add layout image
fig.add_layout_image(labs_logo_dict)

# Update layout with title and dimensions
fig.update_layout(
    title={
        'text': 'Ratio of New Listings / Home Sales (all unit types)',
        'y': 0.98,
        'x': 0.5,
        'xanchor': 'center',
        'yanchor': 'top'
    },
    title_font_color='#FFFFFF',
    width=1200,  # Slightly wider to accommodate more columns
    height=1600,  # Adjust height based on the number of markets
    paper_bgcolor='#080D16',
    margin=dict(l=10, r=10, t=120, b=10)
)

# Show the plot
fig.show()
